# Decorators and context managers

## What are *args and **kwargs?
*args and **kwargs let any method accept any combination of arguments and pass them through untouched.

In [ ]:
# *args collects all positional arguments into a tuple.
def f(*args):
    print(args)

f(1, 2, 3)   # → (1, 2, 3)

In [ ]:
# **kwargs collects all keyword arguments into a dictionary.
def f(**kwargs):
    print(kwargs)

f(a=1, b=2)  # → {'a': 1, 'b': 2}

## Decorators

A decorator lets you wrap a function with extra behavior without changing its code.
You pass a function into another function that returns a modified version of it.

In [ ]:
def log(func):
    def wrapper(*args, **kwargs):
        print(f"Calling {func.__name__}")
        print("Args:", args)
        print("Kwargs:", kwargs)
        return func(*args, **kwargs)
    return wrapper

@log
def greet(name):
    print(f"Hello {name}!")

greet("Alice")


In [ ]:
#timing decorator

import time

def timer(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f"{func.__name__} took {end - start:.4f}s")
        return result
    return wrapper

@timer
def slow_add(a, b):
    time.sleep(0.5)
    return a + b

slow_add(3, 4)


## Context manager

A context manager sets something up before a block of code, and cleans it up afterward.
They are commonly used for managing resources (files, connections, locks).

In [ ]:
class MyContext:
    def __enter__(self):
        print("Entering")
        return self

    def __exit__(self, exc_type, exc, tb):
        print("Exiting")

with MyContext():
    print("Inside")

In [ ]:
from contextlib import contextmanager

@contextmanager
def my_context():
    print("Entering")
    yield
    print("Exiting")

with my_context():
    print("Inside")


## Why use yield instead of return func(*args, **kwargs) in a @contextmanager?
Because yield pauses, while return ends the function.

In [ ]:
@contextmanager
def my_ctx():
    print("Setup")       # __enter__
    yield "value"        # <— pause here
    print("Cleanup")     # __exit__

with my_ctx():
  print("Do something")

In [ ]:
@contextmanager
def my_ctx():
    print("Setup")
    return "value"   # <— function ENDS here
    print("Cleanup") # never runs!

with my_ctx():
  print("Do something")

| Concept      | Decorator                     | Context Manager                       |
| ------------ | ----------------------------- | ------------------------------------- |
| Used for     | Wrapping a **function call**  | Wrapping a **block of code**          |
| When it runs | When the function is called   | When entering/exiting a `with` block  |
| Typical use  | Logging, caching, validation  | Open/close resources, temporary state |
| Syntax       | `@decorator` above a function | `with something:` before a block      |


# Concurrency vs Parallelism

Use concurrency when tasks wait a lot.



Use parallelism when tasks compute a lot.

## Threads (I/O bound)

In [ ]:
import threading
import time

def task(name):
    print(f"{name} start")
    time.sleep(1)  # Simulates I/O
    print(f"{name} end")

t1 = threading.Thread(target=task, args=("A",))
t2 = threading.Thread(target=task, args=("B",))

time_start = time.perf_counter()
t1.start()
t2.start()
t1.join()
t2.join()
time_end = time.perf_counter()
print(time_end-time_start)


## asyncio (cooperative multitasking)

In [ ]:
import asyncio

async def task(name):
    print(f"{name} start")
    await asyncio.sleep(1)  # async wait
    print(f"{name} end")

async def main():
    await asyncio.gather(task("A"), task("B"))

# asyncio.run(main())
await main()


## multiprocessing (CPU-bound)

In [ ]:
from multiprocessing import Process
import time

def compute():
    start = time.time()
    s = sum(i * i for i in range(10_000_000))
    print("Done in:", time.time() - start)

p1 = Process(target=compute)
p2 = Process(target=compute)

p1.start()
p2.start()
p1.join()
p2.join()


| Concept      | Concurrency           | Parallelism            |
| ------------ | --------------------- | ---------------------- |
| How          | Switching             | Running simultaneously |
| Best for     | I/O-bound tasks       | CPU-bound tasks        |
| Python tools | threads / asyncio     | multiprocessing        |
| Limitation   | GIL affects threads   | Higher memory cost     |
| Mental image | One worker multitasks | Many workers at once   |


# Type hints, dataclasses, and Pydantic models

## Type hints

Type hints describe what types your variables and functions expect.

They do not **enforce types at runtime**, but help with:
* readability
* IDE autocomplete
* static checkers (mypy, pyright)



In [ ]:
def add(a: int, b: int) -> int:
    return a + b


In [ ]:
print(add(3,5))

In [ ]:
print(add("a","b"))

## Dataclasses
A dataclass is a lightweight way to create data containers without boilerplate.

Python auto-generates:

* __ init __
*  __ repr __
* __ eq __

In [ ]:
from dataclasses import dataclass

@dataclass
class User:
    name: str
    age: int

    def is_adult(self) -> bool:
        return self.age >= 18

In [ ]:
u = User("Alice", 20)
print(u.is_adult())  # True

## Pydantic models

A Pydantic model is like a dataclass with validation and parsing.

It:
* checks types at runtime
* converts data when possible
* raises clear errors when invalid

In [ ]:
from pydantic import BaseModel

class User(BaseModel):
    name: str
    age: int

u = User(name="Alice", age="30")
print(u)


In [ ]:
u = User("Alice",30)

In [ ]:
User(name="Alice", age="old")

| Feature            | Type hints  | Dataclass     | Pydantic       |
| ------------------ | ----------- | ------------- | -------------- |
| Syntax help        | ✅          | ✅           | ✅             |
| Runtime validation | ❌           | ❌             | ✅             |
| Auto `__init__`    | ❌           | ✅            | ✅             |
| Type coercion      | ❌           | ❌             | ✅             |
| Best for           | APIs, hints | Internal data | External input |


# Performance tuning and profiling

## First rule of performance


> Don't optimize blindly. Measure first.


Python is often fast enough.
Optimize only when you know where time is spent.

## Timing

In [ ]:
# Basic example
import time

start = time.perf_counter()

time.sleep(1)

stop = time.perf_counter()

print(f"Time: {stop-start} s")

In [ ]:
# Basic example
import time

for _ in range(3):

  start = time.perf_counter()

  sum(range(1_000))

  stop = time.perf_counter()

  print(f"Time: {stop-start} s")

In [ ]:
import timeit

time = timeit.timeit(
    "sum(range(1_000))",
    number=10_000
)

# Run the code 10,000 times, then return the total time (in seconds).
print("Total time:", time)
print(f"Time per iteration {time/10_000}")

## Profiling

In [ ]:
import cProfile

def slow():
    s = 0
    for i in range(10_000):
        s += i
    return s

cProfile.run("slow()")


## Built in function

In [ ]:
import timeit


def slow():
  s = 0
  for i in range(10_000):
      s += i
  return s



time_slow = timeit.timeit(
    slow,
    number=10_000
)


time_built_in = timeit.timeit(
    "sum(range(10_000))",
    number=10_000
)

print(f"Time slow: {time_slow/10000} s")
print(f"Time built-in: {time_built_in/10000} s")

## Caching

Caching trades memory for speed by avoiding repeated computation.

In [ ]:
from functools import lru_cache

@lru_cache
def fib(n):
    if n < 2:
        return n
    return fib(n - 1) + fib(n - 2)


def fib_slow(n):
    if n < 2:
        return n
    return fib(n - 1) + fib(n - 2)


import time

start = time.perf_counter()

fib_slow(100)

stop = time.perf_counter()

time_slow = stop-start

start = time.perf_counter()

fib(100)

stop = time.perf_counter()

time_fast = stop-start
print(f"Time slow: {time_slow * 1000} ms")
print(f"Time fast: {time_fast * 1000} ms")


In [ ]:
def square(n):
    print("Computing...")
    return n * n

square(4)
square(4)

In [ ]:
from functools import lru_cache

@lru_cache
def square(n):
    print("Computing...")
    return n * n

square(4)
square(4)


## Memory profiling

Performance is also about memory. Prefer generators over lists for large data.


In [ ]:
import sys

numbers_list = [i for i in range(10_000_000)]

print("List size (bytes):", sys.getsizeof(numbers_list))


In [ ]:
numbers_gen = (i for i in range(10_000_000))

print("Generator size (bytes):", sys.getsizeof(numbers_gen))


In [ ]:
import time
import tracemalloc

N = 2_000_000

def list_version():
    data = [i for i in range(N)]
    return sum(data[:5])

def generator_version():
    data = (i for i in range(N))
    total = 0
    for i in data:
        total += i
        if i == 4:
            break
    return total


# --- List version ---
tracemalloc.start()
t0 = time.perf_counter()
list_result = list_version()
t1 = time.perf_counter()
list_current, list_peak = tracemalloc.get_traced_memory()
tracemalloc.stop()


# --- Generator version ---
tracemalloc.start()
t2 = time.perf_counter()
gen_result = generator_version()
t3 = time.perf_counter()
gen_current, gen_peak = tracemalloc.get_traced_memory()
tracemalloc.stop()


print("List implementation")
print(f"Time: {t1 - t0:.6f} s")
print(f"Peak memory: {list_peak / 1024 / 1024:.2f} MB")

print()
print("Generator implementation")
print(f"Time: {t3 - t2:.6f} s")
print(f"Peak memory: {gen_peak / 1024 / 1024:.2f} MB")
